# Get Consensus from 3 LLMs on CWE Assignment

See https://cybersecai.github.io/Vulnrichment/Vulnrichment/ for context.

To optimize API and token usage and stay within limits, CVE-CWE pairs are processed in batches of batch_size CVE-CWE pairs.

This reduces the 
1. need to send the prompt for each CVE-CWE pair
2. number of calls to the API
   
The number needs to be small enough such that the Rationale from multiple CVE-CWE pairs fits within the output token limit for the LLM.


## LLM Info

### JSON Mode info 
1. https://ai.google.dev/gemini-api/docs/json-mode?lang=python 
2. https://platform.openai.com/docs/guides/json-mode
3. https://github.com/anthropics/anthropic-cookbook/blob/main/misc/how_to_enable_json_mode.ipynb "Claude doesn't have a formal "JSON Mode" with constrained sampling."

### Usage / Plan
1. OpenAI
   1. https://platform.openai.com/account/usage
   2. https://platform.openai.com/docs/guides/rate-limits/usage-tiers?context=tier-one
2. Gemini
   1. https://cloud.google.com/gemini/docs/quotas
3. Claude
   1. https://support.anthropic.com/en/articles/8324991-about-claude-pro-usage
   2. https://support.anthropic.com/en/articles/8325614-how-can-i-maximize-my-claude-pro-usage
   3. https://console.anthropic.com/settings/plans
   4. https://console.anthropic.com/settings/limits
   

### References
1. https://python.langchain.com/v0.1/docs/integrations/chat/openai/
2. https://claude3.pro/claude-3-5-sonnet-with-langchain/
3. https://python.langchain.com/v0.2/docs/integrations/chat/google_generative_ai/

In [1]:
#!pip install langchain_openai langchain_google_genai langchain_anthropic langchain

In [2]:
import pandas as pd
import json
import csv
import os
from datetime import datetime
from tqdm import tqdm

from dotenv import dotenv_values
from dotenv import load_dotenv

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic

from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.schema import AIMessage
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
cwe_only_cisa_adp_input_file_path = './data_out/extracted_cwe_info_cisa_adp_only.csv' # Entries with CWEs only
llm_consensus_file_path = './data_out/llm_consensus.csv'
batch_size=10 # number of entries per batch.  
MAX_WORKERS=3 # threads


In [4]:

# load .env file to environment
load_dotenv()

config = dotenv_values(".env")

# Set up API keys (replace with your actual API keys)
os.environ["OPENAI_API_KEY"] = config['OPENAI_API_KEY']
os.environ["ANTHROPIC_API_KEY"] = config['ANTHROPIC_API_KEY']
os.environ["GOOGLE_API_KEY"] = config['GOOGLE_API_KEY']

#If using env vars
#GOOGLE_API_KEY=config['GOOGLE_API_KEY']
#OPENAI_API_KEY=config['OPENAI_API_KEY']
#ANTHROPIC_API_KEY=config['ANTHROPIC_API_KEY']


In [5]:
# Read the CSV file
try:
    df = pd.read_csv(cwe_only_cisa_adp_input_file_path, quoting=csv.QUOTE_ALL, escapechar='\\') # safe CSV

except FileNotFoundError:
    print("Error: CSV file not found.")
    exit(1)

In [6]:
df=df[:20] # for test purposes take the first N rows only
df

,cve_id,Container,adp_providerMetadata_orgId,adp_providerMetadata_shortName,adp_providerMetadata_dateUpdated,CVE_Description,cna_providerMetadata_orgId,cna_providerMetadata_shortName,cna_providerMetadata_dateUpdated,CWE_ID,CWE_Description
0,CVE-2021-35559,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-06-25T16:05:50.566Z,"Vulnerability in the Java SE, Oracle GraalVM E...",NaN,NaN,NaN,CWE-400,CWE-400 Uncontrolled Resource Consumption
1,CVE-2021-26928,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-05-01T15:18:46.280Z,BIRD through 2.0.7 does not provide functional...,NaN,NaN,NaN,CWE-306,CWE-306 Missing Authentication for Critical Fu...
2,CVE-2021-26918,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-05-01T15:09:54.735Z,The ProBot bot through 2021-02-08 for Discord ...,NaN,NaN,NaN,CWE-434,CWE-434 Unrestricted Upload of File with Dange...
3,CVE-2021-34983,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-05-08T15:08:02.757Z,NETGEAR Multiple Routers httpd Missing Authent...,NaN,NaN,NaN,CWE-120,CWE-120 Buffer Copy without Checking Size of I...
4,CVE-2021-33990,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-07-12T15:32:38.330Z,Liferay Portal 6.2.5 allows Command=FileUpload...,NaN,NaN,NaN,CWE-78,CWE-78 Improper Neutralization of Special Elem...
5,CVE-2021-33161,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-05-22T18:39:19.840Z,Improper input validation in some Intel(R) Eth...,NaN,NaN,NaN,CWE-20,CWE-20 Improper Input Validation
6,CVE-2021-33145,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-05-22T15:07:40.494Z,Uncaught exception in some Intel(R) Ethernet A...,NaN,NaN,NaN,CWE-248,CWE-248 Uncaught Exception
7,CVE-2021-33146,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-05-23T16:14:03.635Z,Improper input validation in some Intel(R) Eth...,NaN,NaN,NaN,CWE-200,CWE-200 Exposure of Sensitive Information to a...
8,CVE-2021-4440,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-06-26T13:57:16.433Z,"In the Linux kernel, the following vulnerabili...",NaN,NaN,NaN,CWE-400,CWE-400 Uncontrolled Resource Consumption
9,CVE-2021-47464,adp,134c704f-9b21-4f2e-91b3-4a467353bcc0,CISA-ADP,2024-06-17T16:08:53.645Z,"In the Linux kernel, the following vulnerabili...",NaN,NaN,NaN,CWE-476,CWE-476 NULL Pointer Dereference


In [7]:
# Set up LangChain for LLM interactions


llms = {
    "claude": ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0,   timeout=None, max_retries=2),
    "gemini": ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0,  timeout=None, max_retries=2, generation_config={"response_mime_type": "application/json"}),
    #"gpt-4o": ChatOpenAI(model="gpt-4o", temperature=0, response_format={ "type": "json_object" })
}

# Define the prompt
# The Rationale string is last as it is most complex

# Update the prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a cybersecurity expert specializing in identifying Common Weakness Enumeration (CWE) IDs from CVE descriptions. Your goal is to analyze if you agree with the assigned CWE ID or not for multiple CVEs."),
    ("human", """
    Analyze the following CVEs and their assigned CWE IDs:
    
    {cve_entries}
    
    For each CVE, output a JSON object containing the following information:
    {{
        "Agree": string, // "Yes" or "No"
        "Confidence": float // a confidence score between 0 and 1
        "Rationale": string, // Only if you do not Agree, provide a rationale why not
    }}
    
    Respond with a JSON array containing an object for each CVE, in the same order as provided.
    """)
])

# Update the output parser
response_schemas = [
    ResponseSchema(name="Agree", description="Whether the model agrees with the assigned CWE ID"),
    ResponseSchema(name="Confidence", description="Confidence score between 0 and 1"),
    ResponseSchema(name="Rationale", description="Rationale for disagreement")
]
#output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


I0000 00:00:1721559508.398094 1239161 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


In [12]:
import json
from datetime import datetime
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

def create_batch_prompt(batch, prompt_template):
    cve_entries = []
    for _, row in batch.iterrows():
        cve_entry = f"""
        CVE ID: {row['cve_id']}
        CVE Description: {row['CVE_Description']}
        Assigned CWE ID: {row['CWE_ID']}
        """
        cve_entries.append(cve_entry)
    
    batch_prompt = prompt_template.format(cve_entries="\n\n".join(cve_entries))
    return batch_prompt

def parse_response(response_content):
    try:
        parsed_responses = json.loads(response_content)
    except json.JSONDecodeError:
        start = response_content.find('[')
        end = response_content.rfind(']') + 1
        if start != -1 and end != -1:
            json_str = response_content[start:end]
            parsed_responses = json.loads(json_str)
        else:
            raise ValueError("Unable to parse response content")
    
    if not isinstance(parsed_responses, list):
        parsed_responses = [parsed_responses]
    
    return parsed_responses

def process_cve_batch(batch, llms, prompt_template, response_dir):
    batch_prompt = create_batch_prompt(batch, prompt_template)
    results = {}
    
    for model_name, llm in llms.items():
        try:
            response = llm.invoke(batch_prompt)
            parsed_responses = parse_response(response.content)
            
            # Save response to file
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{model_name}_response_{timestamp}.json"
            filepath = os.path.join(response_dir, filename)
            with open(filepath, 'w') as f:
                json.dump({
                    'model': model_name,
                    'prompt': batch_prompt,
                    'response': response.content,
                    'parsed_responses': parsed_responses
                }, f, indent=2)
            
            for i, (_, row) in enumerate(batch.iterrows()):
                cve_id = row['cve_id']
                if cve_id not in results:
                    results[cve_id] = {
                        'CVE ID': cve_id,
                        'CVE Description': row['CVE_Description'],
                        'Assigned CWE ID': row['CWE_ID']
                    }
                
                if i < len(parsed_responses):
                    results[cve_id].update({
                        f'{model_name}_Agreement': parsed_responses[i]['Agree'],
                        f'{model_name}_Confidence': parsed_responses[i]['Confidence'],
                        f'{model_name}_Rationale': parsed_responses[i].get('Rationale', ''),
                        f'{model_name}_ResponseFile': filename
                    })
                else:
                    print(f"Warning: Missing response for CVE {cve_id} from {model_name}")
                    results[cve_id].update({
                        f'{model_name}_Agreement': '',
                        f'{model_name}_Confidence': '',
                        f'{model_name}_Rationale': '',
                        f'{model_name}_ResponseFile': filename
                    })
        except Exception as e:
            print(f"Error processing batch with {model_name}: {str(e)}")
            for _, row in batch.iterrows():
                cve_id = row['cve_id']
                if cve_id not in results:
                    results[cve_id] = {
                        'CVE ID': cve_id,
                        'CVE Description': row['CVE_Description'],
                        'Assigned CWE ID': row['CWE_ID']
                    }
                results[cve_id].update({
                    f'{model_name}_Agreement': '',
                    f'{model_name}_Confidence': '',
                    f'{model_name}_Rationale': '',
                    f'{model_name}_ResponseFile': ''
                })
    
    return list(results.values())

def process_cves_in_batches(df, llms, prompt_template, batch_size, max_workers=5, response_dir='responses'):
    results = []
    total_batches = (len(df) + batch_size - 1) // batch_size
    
    # Create response directory if it doesn't exist
    os.makedirs(response_dir, exist_ok=True)
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_batch = {
            executor.submit(
                process_cve_batch, 
                df.iloc[i:i+batch_size], 
                llms, 
                prompt_template, 
                response_dir
            ): i for i in range(0, len(df), batch_size)
        }
        
        for future in tqdm(as_completed(future_to_batch), total=total_batches, desc="Processing CVE Batches"):
            batch_results = future.result()
            results.extend(batch_results)
    
    return pd.DataFrame(results)

# Usage
results_df = process_cves_in_batches(df, llms, prompt_template, batch_size)

Processing CVE Batches:  50%|█████     | 1/2 [00:16<00:16, 16.97s/it]

Processing CVE Batches: 100%|██████████| 2/2 [00:20<00:00, 10.03s/it]


In [13]:
results_df

,CVE ID,CVE Description,Assigned CWE ID,claude_Agreement,claude_Confidence,claude_Rationale,claude_ResponseFile,gemini_Agreement,gemini_Confidence,gemini_Rationale,gemini_ResponseFile
0,CVE-2021-47329,"In the Linux kernel, the following vulnerabili...",CWE-400,Yes,0.9,,claude_response_20240721_120550.json,Yes,0.9,,gemini_response_20240721_120600.json
1,CVE-2021-47368,"In the Linux kernel, the following vulnerabili...",CWE-400,Yes,0.9,,claude_response_20240721_120550.json,No,0.7,The description points to an issue where data ...,gemini_response_20240721_120600.json
2,CVE-2021-47482,"In the Linux kernel, the following vulnerabili...",CWE-544,Yes,0.8,,claude_response_20240721_120550.json,No,0.6,The description highlights improper error hand...,gemini_response_20240721_120600.json
3,CVE-2021-47371,"In the Linux kernel, the following vulnerabili...",CWE-400,Yes,0.9,,claude_response_20240721_120550.json,Yes,0.8,,gemini_response_20240721_120600.json
4,CVE-2021-47238,"In the Linux kernel, the following vulnerabili...",CWE-400,Yes,0.9,,claude_response_20240721_120550.json,Yes,0.9,,gemini_response_20240721_120600.json
5,CVE-2021-47441,"In the Linux kernel, the following vulnerabili...",CWE-787,Yes,0.9,,claude_response_20240721_120550.json,Yes,0.9,,gemini_response_20240721_120600.json
6,CVE-2021-47486,"In the Linux kernel, the following vulnerabili...",CWE-476,Yes,0.9,,claude_response_20240721_120550.json,No,0.7,The description describes a soft lockup (livel...,gemini_response_20240721_120600.json
7,CVE-2021-47274,"In the Linux kernel, the following vulnerabili...",CWE-125,No,0.7,The description mentions a use-after-free issu...,claude_response_20240721_120550.json,Yes,0.9,,gemini_response_20240721_120600.json
8,CVE-2021-47242,"In the Linux kernel, the following vulnerabili...",CWE-667,Yes,0.9,,claude_response_20240721_120550.json,,,,gemini_response_20240721_120600.json
9,CVE-2021-47259,"In the Linux kernel, the following vulnerabili...",CWE-416,No,0.8,The description mentions an out-of-bounds memo...,claude_response_20240721_120550.json,,,,gemini_response_20240721_120600.json


In [8]:
def create_batch_prompt(batch, prompt_template):
    cve_entries = []
    for _, row in batch.iterrows():
        cve_entry = f"""
        CVE ID: {row['cve_id']}
        CVE Description: {row['CVE_Description']}
        Assigned CWE ID: {row['CWE_ID']}
        """
        cve_entries.append(cve_entry)
    
    batch_prompt = prompt_template.format(cve_entries="\n\n".join(cve_entries))
    return batch_prompt

def parse_response(response_content):
    try:
        # First, try to parse the content as JSON
        parsed_responses = json.loads(response_content)
    except json.JSONDecodeError:
        # If JSON parsing fails, try to extract the JSON array from the text
        start = response_content.find('[')
        end = response_content.rfind(']') + 1
        if start != -1 and end != -1:
            json_str = response_content[start:end]
            parsed_responses = json.loads(json_str)
        else:
            raise ValueError("Unable to parse response content")
    
    # Ensure the parsed response is a list
    if not isinstance(parsed_responses, list):
        parsed_responses = [parsed_responses]
    
    return parsed_responses


def process_cve_batch(batch, llms, prompt_template, response_dir):
    batch_prompt = create_batch_prompt(batch, prompt_template)
    results = []
    responses = []
    
    for model_name, llm in llms.items():
        try:
            response = llm.invoke(batch_prompt)
            parsed_responses = parse_response(response.content)
            
            # Save response to file
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{model_name}_response_{timestamp}.json"
            filepath = os.path.join(response_dir, filename)
            with open(filepath, 'w') as f:
                json.dump({
                    'model': model_name,
                    'prompt': batch_prompt,
                    'response': response.content,
                    'parsed_responses': parsed_responses
                }, f, indent=2)
            
            for i, (_, row) in enumerate(batch.iterrows()):
                if i < len(parsed_responses):
                    result = {
                        'CVE ID': row['cve_id'],
                        'CVE Description': row['CVE_Description'],
                        'Assigned CWE ID': row['CWE_ID'],
                        f'{model_name}_Agreement': parsed_responses[i]['Agree'],
                        f'{model_name}_Confidence': parsed_responses[i]['Confidence'],
                        f'{model_name}_Rationale': parsed_responses[i].get('Rationale', ''),
                        f'{model_name}_ResponseFile': filename
                    }
                else:
                    print(f"Warning: Missing response for CVE {row['cve_id']} (CWE {row['CWE_ID']}) from {model_name}")
                    result = {
                        'CVE ID': row['cve_id'],
                        'CVE Description': row['CVE_Description'],
                        'Assigned CWE ID': row['CWE_ID'],
                        f'{model_name}_Agreement': '',
                        f'{model_name}_Confidence': '',
                        f'{model_name}_Rationale': '',
                        f'{model_name}_ResponseFile': filename
                    }
                results.append(result)
                responses.append(response)
        except Exception as e:
            print(f"Error processing batch with {model_name}: {str(e)}")
            for _, row in batch.iterrows():
                results.append({
                    'CVE ID': row['cve_id'],
                    'CVE Description': row['CVE_Description'],
                    'Assigned CWE ID': row['CWE_ID'],
                    f'{model_name}_Agreement': '',
                    f'{model_name}_Confidence': '',
                    f'{model_name}_Rationale': '',
                    f'{model_name}_ResponseFile': ''
                })
    
    return results, responses

def process_cves_in_batches(df, llms, prompt_template, batch_size, max_workers=MAX_WORKERS, response_dir='responses'):
    results = []
    total_batches = (len(df) + batch_size - 1) // batch_size
    
    # Create response directory if it doesn't exist
    os.makedirs(response_dir, exist_ok=True)
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_batch = {
            executor.submit(
                process_cve_batch, 
                df.iloc[i:i+batch_size], 
                llms, 
                prompt_template, 
                response_dir
            ): i for i in range(0, len(df), batch_size)
        }
        
        for future in tqdm(as_completed(future_to_batch), total=total_batches, desc="Processing CVE Batches"):
            batch_results, _ = future.result()
            results.extend(batch_results)
    
    return pd.DataFrame(results)

# Usage
results_df = process_cves_in_batches(df, llms, prompt_template, batch_size)

Processing CVE Batches:   0%|          | 0/189 [00:10<?, ?it/s]


In [ ]:
results_df

,CVE ID,CVE Description,Assigned CWE ID,claude_Agreement,claude_Confidence,claude_Rationale,claude_ResponseFile,gemini_Agreement,gemini_Confidence,gemini_Rationale,gemini_ResponseFile
0,CVE-2021-47329,"In the Linux kernel, the following vulnerabili...",CWE-400,Yes,0.9,,claude_response_20240720_235323.json,NaN,NaN,NaN,NaN
1,CVE-2021-47368,"In the Linux kernel, the following vulnerabili...",CWE-400,Yes,0.9,,claude_response_20240720_235323.json,NaN,NaN,NaN,NaN
2,CVE-2021-47482,"In the Linux kernel, the following vulnerabili...",CWE-544,Yes,0.8,,claude_response_20240720_235323.json,NaN,NaN,NaN,NaN
3,CVE-2021-47371,"In the Linux kernel, the following vulnerabili...",CWE-400,Yes,0.9,,claude_response_20240720_235323.json,NaN,NaN,NaN,NaN
4,CVE-2021-47238,"In the Linux kernel, the following vulnerabili...",CWE-400,Yes,0.9,,claude_response_20240720_235323.json,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3763,CVE-2024-37764,MachForm up to version 19 is affected by an au...,CWE-79,NaN,NaN,NaN,NaN,Yes,0.95,,gemini_response_20240721_001036.json
3764,CVE-2024-37622,Xinhu RockOA v2.6.3 was discovered to contain ...,CWE-79,NaN,NaN,NaN,NaN,Yes,0.95,,gemini_response_20240721_001036.json
3765,CVE-2024-37643,TRENDnet TEW-814DAP v1_(FW1.01B01) was discove...,CWE-121,NaN,NaN,NaN,NaN,Yes,0.9,,gemini_response_20240721_001036.json
3766,CVE-2024-37569,An issue was discovered on Mitel 6869i through...,CWE-77,NaN,NaN,NaN,NaN,Yes,0.95,,gemini_response_20240721_001036.json


In [ ]:

#results_df.to_csv(llm_consensus_file_path, index=False, quoting=csv.QUOTE_ALL, escapechar='\\') # safe CSV

In [ ]:
results_df.claude_Agreement.value_counts()

Yes    1608
No      268
          8
Name: claude_Agreement, dtype: int64

In [ ]:
results_df.gemini_Agreement.value_counts()

Yes    1270
No      459
        155
Name: gemini_Agreement, dtype: int64

In [ ]:
#results_df['gpt-4_Agreement'].value_counts()